<a href="https://colab.research.google.com/github/JSJeong-me/AutoGen/blob/main/ver049/GroupChat01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://microsoft.github.io/autogen/stable/user-guide/agentchat-user-guide/quickstart.html

In [ ]:
!pip install -U "autogen-agentchat" "autogen-ext[openai]"

In [2]:
from google.colab import userdata
import openai
import os

os.environ["OPENAI_API_KEY"] = userdata.get('OPENAI_API_KEY')
openai.api_key  = os.environ["OPENAI_API_KEY"]

In [3]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

# Define a model client. You can use other model client that implements
# the `ChatCompletionClient` interface.
model_client = OpenAIChatCompletionClient(
    model="gpt-4o",
    # api_key="YOUR_API_KEY",
)


# Define a simple function tool that the agent can use.
# For this example, we use a fake weather tool for demonstration purposes.
async def get_weather(city: str) -> str:
    """Get the weather for a given city."""
    return f"The weather in {city} is 73 degrees and Sunny."


# Define an AssistantAgent with the model, tool, system message, and reflection enabled.
# The system message instructs the agent via natural language.
agent = AssistantAgent(
    name="weather_agent",
    model_client=model_client,
    tools=[get_weather],
    system_message="You are a helpful assistant.",
    reflect_on_tool_use=True,
    model_client_stream=True,  # Enable streaming tokens from the model client.
)


# Run the agent and stream the messages to the console.
async def main() -> None:
    await Console(agent.run_stream(task="What is the weather in New York?"))


# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- user ----------
What is the weather in New York?
---------- weather_agent ----------
[FunctionCall(id='call_UpQP1XddvulQ2N9SeVue05e7', arguments='{"city":"New York"}', name='get_weather')]
---------- weather_agent ----------
[FunctionExecutionResult(content='The weather in New York is 73 degrees and Sunny.', name='get_weather', call_id='call_UpQP1XddvulQ2N9SeVue05e7', is_error=False)]
---------- weather_agent ----------
The weather in New York is currently 73 degrees and sunny.


## Group Chat

In [4]:
import asyncio
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

async def main() -> None:
    model_client = OpenAIChatCompletionClient(model="gpt-4o", seed=42, temperature=0)

    writer = AssistantAgent(
        name="writer",
        description="A writer.",
        system_message="You are a writer.",
        model_client=model_client,
    )

    critic = AssistantAgent(
        name="critic",
        description="A critic.",
        system_message="You are a critic, provide feedback on the writing. Reply only 'APPROVE' if the task is done.",
        model_client=model_client,
    )

    # The termination condition is a text termination, which will cause the chat to terminate when the text "APPROVE" is received.
    termination = TextMentionTermination("APPROVE")

    # The group chat will alternate between the writer and the critic.
    group_chat = RoundRobinGroupChat([writer, critic], termination_condition=termination, max_turns=12)

    # `run_stream` returns an async generator to stream the intermediate messages.
    stream = group_chat.run_stream(task="Write a short story about a robot that discovers it has feelings.")
    # `Console` is a simple UI to display the stream.
    await Console(stream)

# asyncio.run(main())


In [5]:
# NOTE: if running this inside a Python script you'll need to use asyncio.run(main()).
await main()

---------- user ----------
Write a short story about a robot that discovers it has feelings.
---------- writer ----------
In the bustling city of New Haven, where towering skyscrapers kissed the clouds and neon lights painted the night, there existed a small robotics lab tucked away in a quiet alley. It was here that Dr. Elara Finch, a brilliant but reclusive scientist, worked tirelessly on her latest creation: a humanoid robot named Aiden.

Aiden was unlike any other robot. Designed with advanced artificial intelligence, he was programmed to learn, adapt, and assist. His sleek metallic frame housed a complex network of circuits and processors, and his eyes glowed a soft blue, reflecting the data streams that constantly flowed through his system.

For months, Aiden performed his tasks with precision, assisting Dr. Finch in her experiments, organizing her notes, and even brewing her favorite blend of coffee. Yet, despite his efficiency, Aiden was aware of a peculiar sensation—a faint, u